In [53]:
import enum
import copy

class Player(enum.Enum):
    x = 1
    o = 2
    
    @property
    def other(self):
        return Player.x if self == Player.o else Player.o

In [55]:
table = {
    None: '.',
    Player.x: 'X',
    Player.o: 'O',
}

class Board():
    def __init__(self):
        self.dimension = 3
        self.grid = [[None for y in range (self.dimension)] for x in range (self.dimension)]
        self.moves = []
    
    
    def print(self):
        print()
        for row in range(self.dimension):
            line = []
            for col in range(self.dimension):
                line.append(table[self.grid[row][col]])
            print('%s' % ("" .join(line)))
    
    
    def make_move(self,x,y,player):
        if(self.is_empty(x,y)):
            self.grid[x][y] = player
            self.moves.append([x,y])
        else:
            raise Exception("Ilegal Move")
    
    def last_move(self):
        return self.moves[-1]
    
    def is_empty(self,x,y):
        return self.grid[x][y] is None
    
    
    def is_valid(self):
        choice = []
        for row in range(self.dimension):
            for col in range(self.dimension):
                if (self.is_empty(row,col)):
                    choice.append([row,col])
        return choice
    
    
    def __deepcopy__(self, memodict={}):
        dp = Board()
        dp.grid = copy.deepcopy(self.grid)
        dp.moves = copy.deepcopy(self.moves)
        return dp
    
    def has_won(self):
        if (len(self.moves) < 5):
            return None

        for row in range(self.dimension):
            win_row = set(self.grid[row])
            
            if (len(win_row) == 1):
                value = win_row.pop()
                if (value != None):
                    return value

        for col in range(self.dimension):
            win_col = set()
            for rol in range(self.dimension):
                win_col.add(self.grid[rol][col])
                
            if (len(win_col) == 1):
                value = win_col.pop()
                if (value != None):
                    return value

        back_dig = set()
        back_dig.add(self.grid[0][0])
        back_dig.add(self.grid[1][1])
        back_dig.add(self.grid[2][2])
        
        if(len(back_dig) == 1):
            value = back_dig.pop()
            if (value != None):
                return value
        
        forw_dig = set()
        forw_dig.add(self.grid[2][0])
        forw_dig.add(self.grid[1][1])
        forw_dig.add(self.grid[0][2])
        
        if(len(forw_dig) == 1):
            value = forw_dig.pop()
            if (value != None):
                return value
        
        return None


In [59]:
class Game():
    def __init__(self,rounds):
        self.rounds = rounds
        self.x_wins = 0
        self.o_wins = 0
        self.ties = 0
    
    def simulation(self,xp,op, print_game = False):
        for _ in range(self.rounds):
            board = Board()
            turn = Player.x

            for x in range(9):
                choice = []

                if (turn == Player.x):
                    choice = xbot.select_move(board)
                else:
                    choice = obot.select_move(board)

                board.make_move(choice[0],choice[1], turn)

                if (print_game):
                    board.print()

                winner = board.has_won()

                if (winner != None):
                    print("congratulations" + str(turn))
                    break
                elif (board.moves == 8):
                    print("it is a tie")
                    break

                turn = turn.other()

        if (winner == Player.x):
            self.x_wins = self.x_wins + 1
        elif (winner == Player.o):
            self.o_wins = self.o_wins + 1
        else:
            self.ties = self.ties + 1
            
                    
        print ("x wins: " + str(self.x_wins))
        print ("o wins: " + str(self.o_wins))
        print ("ties: " + str(self.ties))


In [109]:
import random
import copy
class random_bot():
    def __init__(self,player):
        self.player = player
    def select_move(self,board):
        choices = board.is_valid()
        
        return random.choice(choices)       

    

class Choice():
    def __init__(self,move,value,depth):
        self.move = move
        self.value = value
        self.depth = depth
        
class super_bot():
    def __init__(self,player):
        self.player = player
    
    def minimax(self,board,is_max,player,depth):
        winner = board.has_won()
        
        if (winner == self.player):
            return Choice(board.last_move(),10 - depth, depth)
        elif (winner == self.player.other):
            return Choice(board.last_move(),-10 + depth, depth)
        elif (len(board.moves) == 9):
            return Choice(board.last_move(),0,depth)
        
        pos_choice = []
        all_choice = board.is_valid()
        
        for x in range(len(all_choice)):
            row = all_choice[x][0]
            col = all_choice[x][1]
            
            newboard = copy.deepcopy(board)
            newboard.make_move(row,col,player)
            result = self.minimax(newboard,not is_max,player.other, depth + 1)
            result.move = newboard.last_move()
            pos_choice.append(result)
            
        max_choice = None
        min_choice = None
        max_value = -100
        min_value = 100

        for r in range(len(pos_choice)):
            choice = pos_choice[r]
            if(is_max and choice.value > max_value):
                max_choice = choice
                max_value = choice.value
            elif(not is_max and choice.value < min_value):
                min_choice = choice
                min_value = choice.value

        if (is_max):
            return max_choice
        else:
            return min_choice

    def select_move(self,board):
        choice = self.minimax(board,True,self.player,0)
        return choice.move

In [110]:
game = Game(10)
xbot = super_bot(Player.x)
obot = super_bot(Player.o)
game.simulation(xbot,obot,True)


X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
OOX
XO.

XXO
OOX
XOX
it is a tie

X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
OOX
XO.

XXO
OOX
XOX
it is a tie

X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
OOX
XO.

XXO
OOX
XOX
it is a tie

X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
OOX
XO.

XXO
OOX
XOX
it is a tie

X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
OOX
XO.

XXO
OOX
XOX
it is a tie

X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
OOX
XO.

XXO
OOX
XOX
it is a tie

X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
OOX
XO.

XXO
OOX
XOX
it is a tie

X..
...
...

X..
.O.
...

XX.
.O.
...

XXO
.O.
...

XXO
.O.
X..

XXO
OO.
X..

XXO
OOX
X..

XXO
O